In [2]:
import pymorphy2
import re
from stop_words import get_stop_words

import numpy as np
import pandas as pd
from operator import itemgetter

In [14]:
class NormalAnalyzer:
    __stemmer = pymorphy2.MorphAnalyzer()
    __cache = {}
    
    def norm(self, token):
        token = token.strip()
        
        norm = ''
        if token not in self.__cache:
    
            res = self.__stemmer.parse(token)
            norm = res[0].normal_form
            self.__cache[token] = norm
        else:
            norm = self.__cache[token]
        return norm

In [17]:
class TextUrl():
    def __init__(self):
        self.sentensesDetect = re.compile(r"[^.!?\s][^.!?]*(?:[.!?](?!['\"]?\s|$)[^.!?]*)*[.!?]?['\"]?(?=\s|$)", re.MULTILINE |  re.DOTALL )
        self.wordsDetect = re.compile(u"[А-Яа-я]+")
        
        self.normal_analizer = NormalAnalyzer()
        self.stop_words = get_stop_words('ru')
        
    def sentenseToWords(self, sent):
        words = self.wordsDetect.findall(sent)
        normal_words = []

        # lemmatize words
        for word in words:
            word = self.normal_analizer.norm(word.lower())
            if word not in self.stop_words:
                normal_words.append(word)
        
        return normal_words

In [16]:
news = [u'Трамп ушел с приема на саммите АТЭС и не поговорил с Путиным. Президент США Дональд Трамп ушел с торжественного приема в честь лидеров саммита АТЭС до его окончания и не поговорил с президентом России Владимиром Путиным, сообщает «РИА Новости». Перед уходом Трамп сфотографировался с лидерами других стран',
u'Госдума предложила приравнять зарубежные СМИ к иноагентам. В Госдуме объявили о готовности проработать ответные меры на инициативу США о признании ряда российских СМИ иностранными агентами. Как сообщает «Газета.ру», российские парламентарии хотят внести поправки в закон об НКО. Они собираются добавить туда пункт о зарубежных СМИ.',
u'Путин обсудил жалобы дольщиков с активистами из Челябинска. Президент РФ Владимир Путин поговорил по телефону с активистами-челябинцами по проблемам ГОК и дольщиков и поручил губернатору плотнее заняться этой темой, глава региона принял решение встретиться с гражданами ещё раз, сообщил пресс-секретарь президента РФ Дмитрий Песков.',
u'Глава ЮНЕСКО назвала проблемы, с которыми сталкивается организация. ЮНЕСКО переживает финансовые и политические, но не стратегические сложности, сказала назначенная на пост генерального директора организации экс-министр культуры Франции Одри Азуле. Она отметила, что организации необходимо восстановить доверие мира и стран-членов.',
u'Президент Филиппин на саммите АТЭС признался в убийстве. Президент Филиппин Родриго Дутерте на встрече с соотечественниками во вьетнамском курортном городе Дананге признался, что в 16 лет убил человека ножом в драке. Об этом сообщило РИА Новости со ссылкой на филиппинские СМИ.',
u'Киев анонсировал приезд инспекторов из США в Крым. Многонациональная инспекционная группа из США 13 ноября посетит с визитом Украину, следует из сообщения Министерства обороны Украины. «Американская сторона включила в район инспекции территорию Автономной Республики Крым», — говорится в сообщении ведомства.',
u'Турчинов пообещал расстреливать украинцев за сотрудничество с Россией. Украинцев, сотрудничающих с Россией в оборонной сфере, будут арестовывать и расстреливать, заявил секретарь СНБО Украины Александр Турчинов. Его выступление опубликовал украинский "5 канал" на YouTube',
u'Шойгу рассказал о разгроме последнего оплота террористов в Сирии. Сирийские войска при поддержке наших ВКС взяли последний оплот террористов - город Абу-Кемаль. Об этом министр обороны РФ Сергей Шойгу объявил в пятницу на совместной коллегии военных ведомств России и Беларуси. Свои общие дела и планы генералы обсуждали в Национальном центре управления обороной РФ.',
u'Польский суд счел законным засекреченное решение по судьбе Свиридова. Воеводский административный суд в Варшаве в пятницу рассмотрел дело российского журналиста Леонида Свиридова и счел законным лишение его статуса долговременного резидента Евросоюза.',
u'Пентагон считает космос пространством для ведения боевых действий. Космос превращается для американских Военно-воздушных сил (ВВС) в пространство ведения боевых действий. Об этом заявили в четверг на пресс-конференции в Пентагоне министр и начальник штаба ВВС США Хезер Уилсон и генерал Дэвид Голдфин.']

it = [u'Apple признала, что экран iPhone X плохо реагирует на холод. Компания Apple признала, что экран iPhone X плохо реагирует на холод. При температуре около нуля он некоторое время не реагирует ни на какие прикосновения.',
u'Миллиардер Мильнер намерен отправить экспедицию на спутник Сатурна. Миллиардер отмечает, что миссию к спутнику Сатурна могут отправить «относительно скоро». Этот проект можно будет реализовать, если миссию удастся подготовить с привлечением частного капитала. Мильнер не назвал предположительную сумму, которая может уйти на реализацию проекта.',
u'На космодром Восточный доставили спутники «Канопус-В». Спутники дистанционного зондирования Земли "Канопус-В" № 3 и № 4 были доставлены на космодром Восточный. Об этом сообщили в пресс-службе госкорпорации "Роскосмос".',
u'Касперский прокомментировал доклад WikiLeaks о вирусе ЦРУ. Глава компании «Лаборатория Касперского» Евгений Касперский на своей странице в соцсети Twitter прокомментировал доклад разоблачительного ресурса WikiLeaks о том, что специалисты Центрального разведывательного управления (ЦРУ) США создали специальный код, который позволяет маскировать работу вредоносных программ под продукцию российской компании.',
u'В «Домодедово» начали тестировать новую систему посадки в самолет. В московском аэропорту «Домодедово» проходит тестирование новая технология, которая позволит пассажирам самостоятельно проходить контроль посадки на борт самолета.',
u'Сооснователь Facebook Шон Паркер: Соцсети вредят нашим мозгам. По словам сооснователя Facebook, разработчики прекрасно понимали, что используют слабые места человеческой психики, но всё равно не остановились.',
u'Уфологи обнаружили на комете Чурюмова-Герасименко яйцеобразный НЛО. Международная группа уфологов, изучая снимки кометы Чурюмова-Герасименко 3-летней давности, обнаружила странный объект. Об этом сообщает портал ToDay News Ufa. Специалисты считают, что обнаруженный объект не является частью кометы.',
u'Ученые нашли на поверхности Марса инопланетные иероглифы. Надписи обнаружил на одном из снимков, присланных на Землю аппаратом с поверхности Красной планеты, уфолог Роберт Андерс. По его словам, надписи напоминают японские иероглифы. Они нанесены не только на поверхность грунта, но и на каменистую поверхность скал.',
u'Ученые рассказали о масштабной плазменной буре на Солнце. Обсерватория солнечной динамики американского космического агентства NASA засекла на поверхности Солнца необычную плазменную активность. Ученые отмечают, что такая буря является редчайшим явлением. Специалисты обнаружили на Солнце активную область, окруженную темной нитью.',
u'Samsung анонсировал новый процессор для Samsung Galaxy S9. Компания Samsung сегодня презентовала второе поколение модернизированного процессора Exynos 9810, который будет устанавливаться в Galaxy S9 и Galaxy S9+. По заявлению корейского производителя новый процессор обеспечит не только прирост производительности, но также будет энергоэффективным.']

In [18]:
textUrl = TextUrl()

news_words = []
for news_item in news:
    news_words.append(textUrl.sentenseToWords(news_item))

it_words = []
for it_item in it:
    it_words.append(textUrl.sentenseToWords(it_item))

In [97]:
class NaiveBayes():
    def __init__(self, classes, alpha=1):
        
        self.classes_cnt = len(classes)
        
        vocab = set()
        for cl in classes:
            for text in cl:
                for word in text:
                    vocab.add(word)
        self.len_vocab = len(vocab)
        
        self.p_c = []
        sum_len = np.array([len(cl) for cl in classes]).sum()
        for cl in classes:
            self.p_c.append(float(len(cl)) / sum_len)
        
        cnt_x_c = []
        for cl in classes:
            cl_dict = {}
            for text in cl:
                for word in text:
                    cl_dict[word] = cl_dict.get(word, 0) + alpha
            cnt_x_c.append(cl_dict)
            
        self.p_x_c = []
        for cl_dict in cnt_x_c:
            self.p_x_c.append(self.normolizeDict(cl_dict, alpha))
            
            
    def lenDict(self, d):
        return np.linalg.norm(d.values())

    def normolizeDict(self, d, alpha=1):
        lenDict = self.lenDict(d)
        if lenDict == 0:
            return 0
        return dict(zip(d.keys(), d.values() / (lenDict + alpha * self.len_vocab)))
    
    def predict(self, text):
        predict = []
        for cl in xrange(self.classes_cnt):
            predict_cl = np.log(self.p_c[cl])
#             predict_cl = self.p_c[cl]
            for word in text:
                predict_cl += np.log(self.p_x_c[cl].get(word, 1.0 / self.len_vocab))
#                 predict_cl += self.p_x_c[cl].get(word, 1.0 / self.len_vocab)
            predict.append(predict_cl)
        return predict

In [98]:
NB = NaiveBayes([news_words, it_words])

In [99]:
def print_predict(texts):
    for text in texts:
        weights = NB.predict(text)
        print "weights: " + str(weights)
        print "class: " + str(np.argmax(weights))
        print '_______'

In [100]:
print '**********************************'
print "\tTRAIN NEWS"
print
print_predict(news_words)
    
print 
print '**********************************'
print "\tTRAIN IT"
print
print_predict(it_words)

**********************************
	TRAIN NEWS

weights: [-166.79500613391932, -191.9804153881453]
class: 0
_______
weights: [-193.59491127533826, -203.92293906772625]
class: 0
_______
weights: [-199.9277765535511, -215.81374041373445]
class: 0
_______
weights: [-179.76860845512624, -185.90570888120953]
class: 0
_______
weights: [-144.76566749249073, -155.99767734868459]
class: 0
_______
weights: [-176.95519773836619, -186.06087588192753]
class: 0
_______
weights: [-129.97095793926556, -138.03216949574053]
class: 0
_______
weights: [-236.10886939201012, -251.6413114524772]
class: 0
_______
weights: [-148.75465153905503, -155.99767734868459]
class: 0
_______
weights: [-183.81131116034359, -197.95167722793576]
class: 0
_______

**********************************
	TRAIN IT

weights: [-96.233336617207328, -86.833585282766194]
class: 1
_______
weights: [-179.25569579361922, -169.59917031193459]
class: 1
_______
weights: [-106.90740653557378, -100.09594895381693]
class: 1
_______
weights: [-

In [101]:
test = [u'Одре Азуле утверждена на посту гендиректора ЮНЕСКО. Генеральная конференция ООН по вопросам образования, науки и культуры (ЮНЕСКО) утвердила кандидатуру экс-министра культуры Франции Одре Азуле на пост генерального директора этой международной организации.',
        u'Google выпустил видео о сравнении своего смартфона Pixel 2 и iPhone. По завершении презентации специалисты заявили, что смартфоны Pixel 2 и Pixel XL 2 станут главными конкурентами iPhone 8 и iPhone X от корпорации Apple. Компания Samsung также выпустила ролик со сравнением своих смартфонов со смартфонами корпорации Apple.'
       ]
test_words = []
for test_item in test:
    test_words.append(textUrl.sentenseToWords(test_item))

In [91]:
print '**********************************'
print "\tTEST"
print
print_predict(test_words)

**********************************
	TEST

weights: [-159.27642271568973, -161.96893918847502]
class: 0
_______
weights: [-131.54452137524828, -129.78289034045272]
class: 1
_______
